# 03 - RFM Segmentation

This notebook covers RFM (Recency, Frequency, Monetary) analysis and segmentation.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import sys
sys.path.append('..')
from src.features import calculate_rfm, calculate_rfm_scores, segment_rfm
from src.visualization import plot_rfm_segments

## 1. Calculate RFM Metrics

In [ ]:
# Calculate RFM
# rfm = calculate_rfm(df, customer_id_col='customer_id', date_col='date', amount_col='amount')
# rfm.head()

## 2. RFM Scoring

In [ ]:
# Score customers
# rfm_scored = calculate_rfm_scores(rfm)
# rfm_scored.head()

## 3. Customer Segmentation

In [ ]:
# Assign segments
# rfm_segmented = segment_rfm(rfm_scored)
# rfm_segmented['segment'].value_counts()

## 4. Visualization

In [ ]:
# Plot segments
# plot_rfm_segments(rfm_segmented)